In [8]:
import ipywidgets as widgets
from skimage import io
from skimage.draw import line_nd
from skimage.util import img_as_ubyte
from skimage.exposure import rescale_intensity
from math import sqrt, radians, sin, cos, pi
import matplotlib.pyplot as plt
from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import ExplicitVRLittleEndian
import pydicom._storage_sopclass_uids

class Widgets():
    def __init__(self):
        files = ["CT_ScoutView.jpg", "Kolo.jpg", "Kropka.jpg", "Kwadraty2.jpg", "Paski2.jpg", "SADDLE_PE.JPG", "Shepp_logan.jpg"]
        self.file = widgets.Dropdown(options=files, description="plik")
        display(self.file)
        
        self.alfa = widgets.BoundedFloatText(description="alfa", min=0.01)
        display(self.alfa)
        
        self.n = widgets.BoundedIntText(description="n", min=2, max=1000)
        display(self.n)
        
        self.l = widgets.BoundedFloatText(description="l", min=1)
        display(self.l)
        
        self.name = widgets.Text(description="imie i nazwisko")
        display(self.name)
        
        self.id = widgets.Text(description="pesel")
        display(self.id)

        self.date = widgets.DatePicker(description='data badania')
        display(self.date)
        
        self.comment = widgets.Text(description="komentarz")
        display(self.comment)
        
        self.slider = widgets.IntSlider(description="iteracje", min=0, step=1)
        
data = Widgets()

Dropdown(description='plik', options=('CT_ScoutView.jpg', 'Kolo.jpg', 'Kropka.jpg', 'Kwadraty2.jpg', 'Paski2.j…

BoundedFloatText(value=0.01, description='alfa', min=0.01)

BoundedIntText(value=2, description='n', max=1000, min=2)

BoundedFloatText(value=1.0, description='l', min=1.0)

Text(value='', description='imie i nazwisko')

Text(value='', description='pesel')

DatePicker(value=None, description='data badania')

Text(value='', description='komentarz')

In [9]:
image = io.imread(data.file.value, as_gray=True)

a, b = len(image[0]), len(image)
r = int(sqrt(a * a + b * b) / 2)
display(widgets.BoundedIntText(description="r", value=r, max=r + 1, disabled=True))

img = [[-1 for _ in range(2 * r + 1)] for _ in range(2 * r + 1)]
for y in range(b):
    for x in range(a):
        img[r - b // 2 + y][r - a // 2 + x] = image[y][x]

delta = data.l.value #* 2
display(widgets.BoundedIntText(description="delta", value=delta, max=int(delta) + 1, disabled=True))

data.slider.max = int(360 // data.alfa.value)
data.slider.value = int(360 // data.alfa.value)
display(data.slider)

BoundedIntText(value=282, description='r', disabled=True, max=283)

BoundedIntText(value=90, description='delta', disabled=True, max=91)

IntSlider(value=360, description='iteracje', max=360)

In [ ]:
sinogram = []
rev = [[0 for _ in range(2 * r + 1)] for _ in range(2 * r + 1)]
for scan in range(data.slider.value):
    sinogram.append([])
    emiter = (r * cos(radians(scan * data.alfa.value)) + r, (r * sin(radians(scan * data.alfa.value)) - r)*(-1))
    #print()
    #print(emiter)
    for d in range(data.n.value):
        sinogram[scan].append(0)
        detector = (r * cos(radians(scan * data.alfa.value) + pi - radians(delta / 2) + radians(d * delta) / (data.n.value - 1)) + r, (r * sin(radians(scan * data.alfa.value) + pi - radians(delta / 2) + radians(d * delta) / (data.n.value - 1)) - r)*(-1))
        #print(detector)
        bresenham = line_nd(emiter, detector)
        count = 0
        for i in range(len(bresenham[0])):
            if img[bresenham[0][i]][bresenham[1][i]] != -1:
                sinogram[scan][d] += img[bresenham[0][i]][bresenham[1][i]]
                count += 1
        if count > 0:
            sinogram[scan][d] /= count
        for i in range(len(bresenham[0])):
            if img[bresenham[0][i]][bresenham[1][i]] != -1:
                rev[bresenham[0][i]][bresenham[1][i]] += sinogram[scan][d]
                
fig, ax = plt.subplots(3, 1)
[axi.set_axis_off() for axi in ax.ravel()]
ax[0].imshow(image, cmap='gray')
ax[1].imshow(sinogram, cmap='gray')
ax[2].imshow(rev, cmap='gray')

In [4]:
import numpy as np

def convert_image_to_ubyte(img):
    return img_as_ubyte(rescale_intensity(img, out_range=(0.0, 1.0)))

def save_as_dicom(file_name, img, patient_data):
    img_converted = convert_image_to_ubyte(img)
    
    # Populate required values for file meta information
    meta = Dataset()
    meta.MediaStorageSOPClassUID = pydicom._storage_sopclass_uids.CTImageStorage
    meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
    meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian  

    ds = FileDataset(None, {}, preamble=b"\0" * 128)
    ds.file_meta = meta

    ds.is_little_endian = True
    ds.is_implicit_VR = False

    ds.SOPClassUID = pydicom._storage_sopclass_uids.CTImageStorage
    ds.SOPInstanceUID = meta.MediaStorageSOPInstanceUID
    
    ds.PatientName = patient_data["PatientName"]
    ds.PatientID = patient_data["PatientID"]
    ds.ImageComments = patient_data["ImageComments"]
    

    ds.Modality = "CT"
    ds.SeriesInstanceUID = pydicom.uid.generate_uid()
    ds.StudyInstanceUID = pydicom.uid.generate_uid()
    ds.FrameOfReferenceUID = pydicom.uid.generate_uid()

    ds.BitsStored = 8
    ds.BitsAllocated = 8
    ds.SamplesPerPixel = 1
    ds.HighBit = 7

    ds.ImagesInAcquisition = 1
    ds.InstanceNumber = 1

    ds.Rows, ds.Columns = img_converted.shape

    ds.ImageType = r"ORIGINAL\PRIMARY\AXIAL"

    ds.PhotometricInterpretation = "MONOCHROME2"
    ds.PixelRepresentation = 0

    pydicom.dataset.validate_file_meta(ds.file_meta, enforce_standard=True)

    ds.PixelData = img_converted.tobytes()

    ds.save_as(file_name, write_like_original=False)
    

patient_data = {"PatientName": data.name.value, "PatientID": data.id.value, "ImageComments": str(data.date.value)+";"+data.comment.value}
save_as_dicom("scan.dcm", np.array(rev), patient_data)

AttributeError: 'list' object has no attribute 'dtype'